# Topic Modeling

Outline:
1) LDA
2) BERTopic 
3) LLM
4) sLDA

## 1) LDA

In [2]:
import pandas as pd
# load publishers data from the corresponding file
publishers = pd.read_csv("../data/processed/publishers.csv") 

In [4]:
import matplotlib.pyplot as plt
import re
from wordcloud import WordCloud
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
# compute topic modeling for all publishers (append all the samples)
df_all = pd.concat([pd.read_csv(f"../data/processed/newspapers/sample_{re.sub(r'\\W+','_ ', pub.lower()).strip('_')}.csv") for pub in publishers['publication']], ignore_index=True)

# Tokenization and stopword removal using regex and sklearn stopwords
custom_stopwords = ENGLISH_STOP_WORDS.union({'said', 'mr', 'also'})

def tokenize_and_clean(text):
    # Keep words with 3 or more alphabetic characters
    tokens = re.findall(r'\b[a-z]{3,}\b', str(text).lower())
    return [t for t in tokens if t not in custom_stopwords]

df_all['tokens'] = df_all['article'].apply(tokenize_and_clean)

# Create dictionary and corpus
dictionary = corpora.Dictionary(df_all['tokens'])
dictionary.filter_extremes(no_below=10, no_above=0.5)
corpus = [dictionary.doc2bow(text) for text in df_all['tokens']]

# Train LDA model
num_topics = 10
lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15, random_state=42)

# Plot wordclouds
fig, axes = plt.subplots(2, 5, figsize=(20, 10), constrained_layout=True)
axes = axes.flatten()

for idx, ax in enumerate(axes):
    topic_words = dict(lda_model.show_topic(idx, 50))
    wc = WordCloud(width=500, height=300, background_color='white', max_words=50)
    wc.generate_from_frequencies(topic_words)
    ax.imshow(wc, interpolation='bilinear')
    ax.set_title(f'Topic {idx + 1}', fontsize=14)
    ax.axis('off')

plt.suptitle('LDA Topics – Word Clouds', fontsize=18)
plt.show()

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# Save LDA model and dictionary
lda_model.save("../models/topic_model/lda_model.gensim")
dictionary.save("../models/topic_model/lda_dictionary.dict")

In [ ]:
from gensim import corpora, models

# Load model and dictionary
lda_model = models.LdaModel.load("../models/topic_model/lda_model.gensim")
dictionary = corpora.Dictionary.load("../models/topic_model/lda_dictionary.dict")

In [ ]:
from gensim.models import CoherenceModel

coherence_model = CoherenceModel(model=lda_model, texts=df_all['tokens'], dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model.get_coherence()
print(f'Coherence Score (c_v): {coherence_score:.4f}')
# A score above 0.4 is generally considered good for topic coherence.

In [ ]:
from datetime import datetime
import re

# --- Assumes these exist ---
# - df['tokens'] = list of preprocessed word tokens
# - df['date'] = parsed datetime
# - df['publication'] = publisher name
# - lda_model = trained gensim LdaModel
# - dictionary = gensim Dictionary used to train the model

# Load all samples from new processed newspapers folder
df_all = pd.concat([pd.read_csv(f"../data/processed/newspapers/sample_{re.sub(r'\\W+','_ ', pub.lower()).strip('_')}.csv") for pub in publishers['publication']], ignore_index=True)

# Step 1: Convert tokens to bag-of-words
corpus = [dictionary.doc2bow(text) for text in df_all['tokens']]

# Step 2: Get topic distribution for each article
def get_topic_dist(bow):
    # Return full-length vector with zero entries where necessary
    dist = lda_model.get_document_topics(bow, minimum_probability=0)
    return [prob for _, prob in dist]

df_all['topic_distribution'] = [get_topic_dist(doc) for doc in corpus]

# Step 3: Unpack topic distributions into separate columns
num_topics = lda_model.num_topics
topic_cols = [f'topic_{i}' for i in range(num_topics)]
df_topics = pd.DataFrame(df_all['topic_distribution'].tolist(), columns=topic_cols)

# Step 4: Combine with metadata
df_meta = df_all[['date', 'publication']].copy()
df_combined = pd.concat([df_meta, df_topics], axis=1)
df_combined['month'] = pd.to_datetime(df_combined['date'], format='mixed', errors='coerce').dt.to_period('M')

# Step 5: Aggregate topic shares by month and publisher
df_monthly_pub = df_combined.groupby(['month', 'publication'])[topic_cols].mean().reset_index()

# Step 6: Save to CSV
df_monthly_pub.to_csv('../data/processed/monthly_topic_shares_by_publisher.csv', index=False)

print("Saved: 'monthly_topic_shares_by_publisher.csv'")

In [3]:
import pandas as pd
import numpy as np
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# --- 1. Load publisher article data ---
# Load publishers
publishers = pd.read_csv("../data/processed/publishers.csv")

# Define safe filename generator
def safe_filename(pub):
    return re.sub(r'\W+', '_', pub.lower()).strip('_')

# Concatenate all article samples
df_all = pd.concat([
    pd.read_csv(f"../data/processed/newspapers/sample_{safe_filename(pub)}.csv")
    for pub in publishers['publication']
], ignore_index=True)

# --- 2. Define keyword filter ---
topic_keywords = [
    "manufacturing", "factory", "production", "industry", "output",
    "supply chain", "logistics", "transport", "shortage"
]

# --- 3. Clean and filter text ---
def clean_text(text_input):
    text_input = str(text_input).lower()
    text_input = re.sub(r'\d+', '', text_input)
    text_input = re.sub(f"[{re.escape(string.punctuation)}]", '', text_input)
    text_input = re.sub(r'\s+', ' ', text_input).strip()
    return text_input

def filter_keywords(text, keywords):
    text = text.lower()
    return ' '.join([
        word for word in text.split()
        if any(k in word for k in keywords)
    ])

# --- 4. Aggregate articles per month ---
df_all['date'] = pd.to_datetime(df_all['date'], errors='coerce')
df_all['month'] = df_all['date'].dt.to_period('M').dt.to_timestamp()

df_monthly = df_all.groupby('month')['article'].apply(lambda x: ' '.join(x.dropna())).reset_index()

# --- 5. Apply keyword filtering ---
df_monthly['filtered_content'] = df_monthly['article'].apply(lambda x: filter_keywords(x, topic_keywords))

# --- 6. Load IPI data and merge ---
df_indpro = pd.read_csv('https://fred.stlouisfed.org/graph/fredgraph.csv?id=INDPRO', parse_dates=['observation_date'])
df_indpro.rename(columns={'observation_date': 'date', 'INDPRO': 'ipi'}, inplace=True)
df_indpro['month'] = df_indpro['date'].dt.to_period('M').dt.to_timestamp()

df_keywords_monthly = pd.merge(df_monthly, df_indpro[['month', 'ipi']], on='month', how='inner')

# --- 7. TF-IDF Vectorization ---
vectorizer = TfidfVectorizer(
    max_df=0.95,
    min_df=10,
    stop_words='english',
    token_pattern=r'\b[a-zA-Z]{3,}\b',
    preprocessor=clean_text
)

X_tfidf = vectorizer.fit_transform(df_keywords_monthly['filtered_content'].fillna(''))

df_features = pd.DataFrame(
    X_tfidf.toarray(),
    columns=[f"kw_{word}" for word in vectorizer.get_feature_names_out()]
)
df_features['month'] = df_keywords_monthly['month'].values
df_features['ipi'] = df_keywords_monthly['ipi'].values

# --- 8. Save final DataFrame ---
df_features.to_csv("../data/processed/df_industry_keywords_monthly.csv", index=False)


FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/newspapers/sample_reuters.csv'

## 2) BERTopic

Goal: Topic shares by month-publisher using BERTopic for the topic modeling. 

In [1]:
import pandas as pd
import re
import os
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from difflib import get_close_matches
from tqdm import tqdm

# Load publishers
publishers = pd.read_csv("../data/processed/publishers.csv")

# Get all sample files
data_dir = "../data/processed/newspapers/"
available_files = os.listdir(data_dir)

# Extract clean basenames
available_basenames = {
    re.sub(r'^sample_|\.csv$', '', fname): fname
    for fname in available_files
    if fname.startswith("sample_") and fname.endswith(".csv")
}

# Helper to sanitize
def sanitize(pub):
    return re.sub(r'\W+', '_', pub.lower()).strip('_')

# Load and concat all files
dfs = []
for pub in tqdm(publishers['publication'], desc="Loading data"):
    pub_clean = sanitize(pub)
    match = get_close_matches(pub_clean, available_basenames.keys(), n=1, cutoff=0.7)
    if match:
        matched_filename = os.path.join(data_dir, available_basenames[match[0]])
        dfs.append(pd.read_csv(matched_filename))
    else:
        print(f"No file found for: {pub}")

df_all = pd.concat(dfs, ignore_index=True)

# Clean articles
def clean_text(text):
    tokens = re.findall(r'\b[a-z]{3,}\b', str(text).lower())
    return ' '.join(tokens)

df_all['clean_article'] = df_all['article'].astype(str).apply(clean_text)
df_all = df_all[df_all['clean_article'].str.strip().astype(bool)].reset_index(drop=True)


Loading data: 100%|██████████| 26/26 [00:08<00:00,  3.04it/s]


In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from bertopic import BERTopic
from sklearn.cluster import MiniBatchKMeans
from umap import UMAP
from sentence_transformers import SentenceTransformer
import numpy as np

# Clean up text data
texts = df_all['clean_article'].tolist()

# Embedding model
embedding_model = SentenceTransformer("paraphrase-MiniLM-L3-v2")

print("Generating embeddings...")
embeddings = embedding_model.encode(
    texts,
    show_progress_bar=True,
    batch_size=256
)

# UMAP and KMeans
umap_model = UMAP(n_components=5, random_state=42)
kmeans_model = MiniBatchKMeans(n_clusters=10, random_state=42) # choose number of clusters based for easier comparison with LDA

# Initialize BERTopic
topic_model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=kmeans_model,
    embedding_model=None,
    language="english",
    calculate_probabilities=True,
    verbose=True
)

# Fit and transform
print("Fitting BERTopic...")
topics, probs = topic_model.fit_transform(texts, embeddings)

# Save results
topic_model.save("../models/topic_model/bertopic_model")
np.save("../models/topic_model/bertopic_embeddings.npy", embeddings)


Generating embeddings...


Batches:   0%|          | 0/995 [00:00<?, ?it/s]

2025-07-10 17:08:01,835 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


Fitting BERTopic...


2025-07-10 17:11:34,071 - BERTopic - Dimensionality - Completed ✓
2025-07-10 17:11:34,075 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-07-10 17:11:34,522 - BERTopic - Cluster - Completed ✓
2025-07-10 17:11:34,550 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-07-10 17:12:17,739 - BERTopic - Representation - Completed ✓
2025-07-10 17:12:18,499 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [3]:
# Get number of real topics
topic_ids = sorted([t for t in topic_model.get_topics().keys() if t != -1])
topic_cols = [f"topic_{i}" for i in topic_ids]

df_meta = df_all[['date', 'publication']].reset_index(drop=True)
df_combined = df_meta.copy()
df_combined['topic'] = topics

# Extract month
df_combined['month'] = pd.to_datetime(df_combined['date'], format='mixed', errors='coerce').dt.to_period('M')

# One-hot encode topics
df_onehot = pd.get_dummies(df_combined['topic'], prefix='topic')

# Combine with meta
df_combined = pd.concat([df_combined[['month', 'publication']], df_onehot], axis=1)

# Group by month + publisher and average (which is topic share)
df_monthly_pub = df_combined.groupby(['month', 'publication']).mean().reset_index()

# Save result
df_monthly_pub.to_csv('../data/processed/monthly_topic_shares_by_publisher_bertopic.csv', index=False)
print("Saved: 'monthly_topic_shares_by_publisher_bertopic.csv'")


Saved: 'monthly_topic_shares_by_publisher_bertopic.csv'


## 3) LLM

In [4]:
# Imports
import os
import re
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from difflib import get_close_matches
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer

# Load and clean data
publishers = pd.read_csv("../data/processed/publishers.csv")
data_dir = "../data/processed/newspapers/"
available_files = os.listdir(data_dir)

available_basenames = {
    re.sub(r'^sample_|\.csv$', '', fname): fname
    for fname in available_files
    if fname.startswith("sample_") and fname.endswith(".csv")
}

def sanitize(pub):
    return re.sub(r'\W+', '_', pub.lower()).strip('_')

dfs = []
for pub in tqdm(publishers['publication'], desc="Loading data"):
    pub_clean = sanitize(pub)
    match = get_close_matches(pub_clean, available_basenames.keys(), n=1, cutoff=0.7)
    if match:
        dfs.append(pd.read_csv(os.path.join(data_dir, available_basenames[match[0]])))
    else:
        print(f"No file found for: {pub}")

df_all = pd.concat(dfs, ignore_index=True)

# Clean text
def clean_text(text):
    tokens = re.findall(r'\b[a-z]{3,}\b', str(text).lower())
    custom_stopwords = {'said', 'mr', 'also'}
    return ' '.join([t for t in tokens if t not in custom_stopwords])

df_all['clean_text'] = df_all['article'].astype(str).apply(clean_text)
df_all = df_all[df_all['clean_text'].str.split().str.len() > 20].reset_index(drop=True)

# Batched Embeddings
print("Generating text embeddings in batches...")
embed_model = SentenceTransformer('paraphrase-MiniLM-L3-v2')
texts = df_all['clean_text'].tolist()
batch_size = 10000
all_embeddings = []

for i in tqdm(range(0, len(texts), batch_size), desc="Embedding batches"):
    batch = texts[i:i+batch_size]
    embeddings = embed_model.encode(batch, show_progress_bar=True)
    all_embeddings.append(embeddings)

embeddings = np.vstack(all_embeddings)

# KMeans clustering
num_topics = 10
kmeans = KMeans(n_clusters=num_topics, random_state=42)
df_all['topic_cluster'] = kmeans.fit_predict(embeddings)

# Load TinyLlama locally
print("Loading TinyLlama model...")
device = 'cuda' if torch.cuda.is_available() else ('mps' if torch.backends.mps.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0").to(device)

topic_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if device == 'cuda' else -1
)

# Generate topic label per cluster
def generate_cluster_topic(texts):
    sample_text = "\n\n".join(texts)
    prompt = f"""Given the following news article excerpts, identify the main shared topic.  
Respond with only 1–3 words (e.g., 'climate change', 'US politics', etc.).

Example:
Articles:
Article 1: The president signed a new climate treaty...
Article 2: World leaders gathered to address emissions...
Topic: climate change

Now do the same for these articles:
{sample_text}

Topic:"""
    try:
        output = topic_pipe(prompt, max_new_tokens=30, temperature=0.7, top_k=40, do_sample=True)
        generated = output[0]["generated_text"]
        return generated.split("Topic:")[-1].strip().split("\n")[0]
    except Exception as e:
        return f"Error: {e}"

print("Generating topic labels per cluster...")
llama_topic_labels = {}
n_samples_per_cluster = 10

for cluster_id in range(num_topics):
    cluster_docs = df_all[df_all['topic_cluster'] == cluster_id]['clean_text']
    if not cluster_docs.empty:
        samples = [" ".join(doc.split()[:50]) for doc in cluster_docs.sample(n=min(n_samples_per_cluster, len(cluster_docs)), random_state=42)]
        topic_label = generate_cluster_topic(samples)
        llama_topic_labels[cluster_id] = topic_label
        print(f"Cluster {cluster_id}: {topic_label}")
    else:
        llama_topic_labels[cluster_id] = "No documents"

# Topic distribution per document
print("Calculating topic distributions...")
similarities = cosine_similarity(embeddings, kmeans.cluster_centers_)
topic_cols = [f'topic_{i}' for i in range(num_topics)]
df_topic_dist = pd.DataFrame(similarities, columns=topic_cols)
df_all = pd.concat([df_all, df_topic_dist], axis=1)

# Temporal aggregation
print("Aggregating over time and publisher...")
df_all['date'] = pd.to_datetime(df_all['date'], errors='coerce')
df_all['month'] = df_all['date'].dt.to_period('M')

df_combined = pd.concat([df_all[['month', 'publication']], df_all[topic_cols]], axis=1)
df_monthly_pub = df_combined.groupby(['month', 'publication'])[topic_cols].mean().reset_index()

# Save
df_monthly_pub.to_csv('../data/processed/monthly_topic_shares_by_publisher_tinyllama.csv', index=False)
print("Saved: 'monthly_topic_shares_by_publisher_tinyllama.csv'")

# Print cluster topics
print("\nFinal Topic Labels:")
for cid, label in llama_topic_labels.items():
    print(f"Topic {cid}: {label}")


Loading data: 100%|██████████| 26/26 [00:08<00:00,  3.03it/s]


Generating text embeddings in batches...


Embedding batches:   0%|          | 0/24 [00:00<?, ?it/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Embedding batches:   4%|▍         | 1/24 [00:22<08:32, 22.28s/it]

...

Batches:   0%|          | 0/281 [00:00<?, ?it/s]

Embedding batches: 100%|██████████| 24/24 [10:09<00:00, 25.40s/it]


Loading TinyLlama model...


Device set to use mps:0


Generating topic labels per cluster...
Cluster 0: criminal justice reform
Cluster 1: fashion
Cluster 2: President Trump and the Paris Agreement
Cluster 3: artificial intelligence
Cluster 4: politics
Cluster 5: technology
Cluster 6: COVID-19
Cluster 7: climate change
Cluster 8: the environment.
Cluster 9: protests
Calculating topic distributions...
Aggregating over time and publisher...
Saved: 'monthly_topic_shares_by_publisher_tinyllama.csv'

Final Topic Labels:
Topic 0: criminal justice reform
Topic 1: fashion
Topic 2: President Trump and the Paris Agreement
Topic 3: artificial intelligence
Topic 4: politics
Topic 5: technology
Topic 6: COVID-19
Topic 7: climate change
Topic 8: the environment.
Topic 9: protests


## 4) Supervised LDA (sLDA) Implementation with Tomotopy

Unlike the previous approach that used a two-stage method (sklearn LDA + Linear Regression), we'll implement true supervised LDA using tomotopy, which integrates the supervision signal (IPI data) directly into the topic modeling process.

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from datetime import datetime
import tomotopy as tp  # We're using tomotopy for supervised LDA
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

### Data Preparation

Let's load the pre-processed newspaper data and the IPI (supervision) data for sLDA modeling.

In [2]:
# Load publishers information
publishers = pd.read_csv("../data/processed/publishers.csv")
print(f"Loaded information for {len(publishers)} publishers")

# Load all samples from processed newspapers folder
# Create a function to convert publisher names to filenames
def publisher_to_filename(publisher_name):
    # Replace special characters with underscores
    return f"sample_{re.sub(r'\W+', '_', publisher_name.lower()).strip('_')}.csv"

# Load all sample files
df_all = pd.concat([
    pd.read_csv(f"../data/processed/newspapers/{publisher_to_filename(pub)}")
    for pub in publishers['publication']
], ignore_index=True)

print(f"Loaded {len(df_all)} newspaper articles")
print(f"Sample data columns: {df_all.columns.tolist()}")

# Examine the first rows to understand the data
df_all.head()

Loaded information for 26 publishers
Loaded 254701 newspaper articles
Sample data columns: ['date', 'year', 'month', 'day', 'author', 'title', 'article', 'url', 'section', 'publication']


,date,year,month,day,author,title,article,url,section,publication
0,2018-06-13,2018,6.0,13,NaN,Volkswagen CEO says still trying to overcome d...,BERLIN (Reuters) - Volkswagen’s (VOWG_p.DE) ne...,https://www.reuters.com/article/us-volkswagen-...,Environment,Reuters
1,2018-11-09,2018,11.0,9,"Jalil Ahmad Rezayee, Storay Karimi",Displaced victims of worst Afghan drought in y...,"HERAT, Afghanistan (Reuters) - Mahiuddin, a 35...",https://www.reuters.com/article/us-afghanistan...,Environment,Reuters
2,2020-03-11,2020,3.0,11,NaN,UK announces five billion pound emergency resp...,LONDON (Reuters) - Finance minister Rishi Suna...,https://www.reuters.com/article/us-britain-bud...,Business News,Reuters
3,2016-11-02,2016,11.0,2,NaN,Fitch: No Impact on Ardshinbank's Ratings from...,(The following statement was released by the r...,http://www.reuters.com/article/idUSFit978977,Intel,Reuters
4,2019-10-15,2019,10.0,15,NaN,U.S. to push diplomacy in bid to halt Syria at...,"WASHINGTON, Oct 15 (Reuters) - The United Stat...",https://www.reuters.com/article/syria-security...,Market News,Reuters


In [3]:
# Load IPI (response variable) data - this is monthly data that we'll use for supervision
df_ipi = pd.read_csv("../data/processed/ipi_data.csv")

# Display the IPI data
print(f"Loaded IPI data with {len(df_ipi)} records")
print("Columns in df_ipi:", df_ipi.columns.tolist())
df_ipi.head()

Loaded IPI data with 3432 records
Columns in df_ipi: ['month', 'publication', 'ipi_value']


,month,publication,ipi_value
0,2010-01,Reuters,89.1897
1,2010-01,The New York Times,89.1897
2,2010-01,CNBC,89.1897
3,2010-01,The Hill,89.1897
4,2010-01,People,89.1897


### Merge newspaper data with IPI data

Since IPI data is monthly while newspaper text is at article level, we need to merge them to assign the monthly IPI values to each article in the corresponding month and publication. The newspaper data uses YYYY-MM-DD format for dates while the IPI data uses YYYY-MM format.

In [4]:
# Create month_key in df_all to match df_ipi format
# Remove time component if present and convert to month_key
df_all['month_key'] = pd.to_datetime(df_all['date'].str.split().str[0]).dt.strftime('%Y-%m')

# Merge the dataframes on month_key and publication
df_merged = pd.merge(
    df_all, 
    df_ipi[['month', 'publication', 'ipi_value']], 
left_on=['month_key', 'publication'],
    right_on=['month', 'publication']
)

# Consolidate 'month' column:
# If 'month_y' (from df_ipi) exists, rename it to 'month'.
# If 'month_x' (from df_all) also exists, it's likely redundant with 'month_key' or not the IPI-aligned month, so drop it.
if 'month_y' in df_merged.columns:
    df_merged.rename(columns={'month_y': 'month'}, inplace=True)
    if 'month_x' in df_merged.columns:
        df_merged.drop(columns=['month_x'], inplace=True, errors='ignore')
elif 'month' not in df_merged.columns:
    # This case should ideally not happen if the merge is successful and df_ipi has 'month'
    print("Warning: 'month' column is missing after merge and suffix handling.")


print("Columns in df_merged after merge and rename:", df_merged.columns.tolist())

print(f"Merged data shape: {df_merged.shape}")
print("\nSample of merged data:")
print(df_merged[['date', 'publication', 'month_key', 'ipi_value']].head())

Columns in df_merged after merge and rename: ['date', 'year', 'day', 'author', 'title', 'article', 'url', 'section', 'publication', 'month_key', 'month', 'ipi_value']
Merged data shape: (254701, 12)

Sample of merged data:
         date publication month_key  ipi_value
0  2018-06-13     Reuters   2018-06   103.2256
1  2018-11-09     Reuters   2018-11   104.0126
2  2020-03-11     Reuters   2020-03    97.6060
3  2016-11-02     Reuters   2016-11    98.3443
4  2019-10-15     Reuters   2019-10   101.5878


### Model Training & Evaluation

Now we'll implement supervised LDA using tomotopy's SLDAModel, which directly incorporates the IPI values during topic inference. The `vars` parameter in SLDAModel specifies that we have one continuous response variable (the normalized IPI value).

In [ ]:
# Define number of topics
num_topics = 10

# Step 1: Text preprocessing for tomotopy
print("Preprocessing text data...")

# Check if 'article' or 'content' column exists
text_column = 'article' if 'article' in df_merged.columns else 'content'

# Define a list of stopwords (standard English stopwords + common newspaper words)
stopwords = set(['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", 
                 "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 
                 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 
                 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 
                 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 
                 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 
                 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 
                 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 
                 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 
                 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 
                 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 
                 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 
                 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 
                 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 
                 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', 
                 "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 
                 'wouldn', "wouldn't", 'said', 'mr', 'also'])

# Tokenization function for tomotopy
def preprocess_text(text):
    # Keep words with 3 or more alphabetic characters
    tokens = re.findall(r'\b[a-z]{3,}\b', str(text).lower())
    return [t for t in tokens if t not in stopwords]

# Apply preprocessing to get tokenized documents
docs_tokens = df_merged[text_column].apply(preprocess_text).tolist()

# Get the IPI values as supervision labels
ipi_values = df_merged['ipi_value'].tolist()

# Normalize IPI values to smaller range for better numerical stability
# Scale to 0-1 range which works better with tomotopy
ipi_min = min(ipi_values)
ipi_max = max(ipi_values)
ipi_values_norm = [(val - ipi_min) / (ipi_max - ipi_min) for val in ipi_values]
print(f"Normalized IPI values range: {min(ipi_values_norm)} to {max(ipi_values_norm)}")

# After tokenization: check how many documents have tokens
num_docs_with_tokens = sum(1 for tokens in docs_tokens if tokens)
print(f"Documents with tokens: {num_docs_with_tokens} / {len(docs_tokens)}")
print("Sample tokenized docs:", docs_tokens[:5])


# Step 2: Initialize and train the supervised LDA model
print(f"Training sLDA model with {num_topics} topics...")

# Initialize the sLDA model with hyperparameters
# For the 'vars' parameter in tomotopy:
# - 1: Use the value 1 for continuous variables
# - Integer > 1: Number of categories for categorical variables
slda_model = tp.SLDAModel(
    k=num_topics,  # Number of topics
    alpha=0.1,     # Prior document-topic density
    eta=0.01,      # Prior topic-word density
    seed=42,       # For reproducibility
    vars=['l']       # One continuous regression variable (the IPI value) - Reverted to 'l'
)

# Add documents with their corresponding labels (IPI values)
for i, (tokens, ipi) in enumerate(zip(docs_tokens, ipi_values_norm)):
    if tokens:  # Ensure document has tokens after preprocessing
        slda_model.add_doc(tokens, [float(ipi)])  # tomotopy expects label as a list, ensure float type

# After adding documents: check the number of documents in the model
print(f"Number of documents in sLDA model: {len(slda_model.docs)}")
assert len(slda_model.docs) > 0, "No documents were added to the sLDA model!"

# Train the model
print("Training sLDA model...")
for i in range(0, 100, 10):
    slda_model.train(10)  # Train 10 iterations at a time
    print(f'Iteration: {i+10}\tLog-likelihood: {slda_model.ll_per_word}')

print("sLDA model training complete")

# Evaluate the model's predictive performance - tomotopy will give us regression coefficients
raw_coef_output = slda_model.get_regression_coef() # Expected: [[c0, c1, ..., c_N-1]] or [c0, ..., c_N-1]

# Correctly extract 1D list of coefficients
if isinstance(raw_coef_output, (list, np.ndarray)) and \
   len(raw_coef_output) == 1 and \
   isinstance(raw_coef_output[0], (list, np.ndarray)) and \
   all(isinstance(c, (int, float, np.number)) for c in raw_coef_output[0]):
    # Handles [[c0, c1, ..., cN-1]]
    coefficients_1d = [float(c) for c in raw_coef_output[0]]
elif isinstance(raw_coef_output, (list, np.ndarray)) and \
     all(isinstance(c, (int, float, np.number)) for c in raw_coef_output):
    # Handles [c0, c1, ..., cN-1] (already 1D)
    coefficients_1d = [float(c) for c in raw_coef_output]
else:
    print(f"Warning: Unexpected format for regression coefficients. Got: {raw_coef_output}")
    coefficients_1d = [] # Fallback

# Define intercept and topic_coefs
# Assuming if len(coefficients_1d) == num_topics, all are topic_coefs and intercept is 0
# If len(coefficients_1d) == num_topics + 1, first is intercept, rest are topic_coefs
if len(coefficients_1d) == num_topics:
    intercept = 0.0
    topic_coefs = coefficients_1d
    print(f"Interpreting {len(coefficients_1d)} coefficients as all topic-specific, intercept = 0.0")
elif len(coefficients_1d) == num_topics + 1:
    intercept = coefficients_1d[0]
    topic_coefs = coefficients_1d[1:]
    print(f"Interpreting {len(coefficients_1d)} coefficients as intercept + {len(topic_coefs)} topic-specific.")
else:
    print(f"Warning: Unexpected number of coefficients ({len(coefficients_1d)}). Expected {num_topics} or {num_topics + 1}. Defaulting intercept to 0 and using all as topic_coefs if available.")
    intercept = 0.0
    topic_coefs = coefficients_1d if coefficients_1d else []


print("\nRegression coefficients (topic influence on IPI):")
print(f"Intercept: {intercept:.4f}")
print(f"Number of topic coefficients: {len(topic_coefs)} (should be {num_topics})")
if len(topic_coefs) != num_topics:
    print(f"WARNING: Number of topic coefficients ({len(topic_coefs)}) does not match num_topics ({num_topics})! Check model initialization and document addition.")
for k, coef in enumerate(topic_coefs):
    print(f"Topic {k}: {coef:.4f}")


# Get topic distributions for validation set to evaluate model performance
# For sLDA, we need to calculate predictions and compare with actual values
print("\nCalculating model predictions...")

# Get regression coefficients from sLDA model
raw_coef_output_pred = slda_model.get_regression_coef() # Renamed to avoid conflict if any subtle scope issue

# Correctly extract 1D list of coefficients for prediction
if isinstance(raw_coef_output_pred, (list, np.ndarray)) and \
   len(raw_coef_output_pred) == 1 and \
   isinstance(raw_coef_output_pred[0], (list, np.ndarray)) and \
   all(isinstance(c, (int, float, np.number)) for c in raw_coef_output_pred[0]):
    # Handles [[c0, c1, ..., cN-1]]
    coefficients_1d_pred = [float(c) for c in raw_coef_output_pred[0]]
elif isinstance(raw_coef_output_pred, (list, np.ndarray)) and \
     all(isinstance(c, (int, float, np.number)) for c in raw_coef_output_pred):
    # Handles [c0, c1, ..., cN-1] (already 1D)
    coefficients_1d_pred = [float(c) for c in raw_coef_output_pred]
else:
    print(f"Warning: Unexpected format for regression coefficients during prediction. Got: {raw_coef_output_pred}")
    coefficients_1d_pred = [] # Fallback

# Define intercept and topic_coefs for prediction (mirroring the logic above)
if len(coefficients_1d_pred) == num_topics:
    intercept_pred = 0.0
    topic_coefs_pred = coefficients_1d_pred
elif len(coefficients_1d_pred) == num_topics + 1:
    intercept_pred = coefficients_1d_pred[0]
    topic_coefs_pred = coefficients_1d_pred[1:]
else: # Fallback, should match the logic for non-pred variables
    intercept_pred = 0.0
    topic_coefs_pred = coefficients_1d_pred if coefficients_1d_pred else []

# Ensure intercept and topic_coefs used in this prediction block are the _pred versions
intercept = intercept_pred
topic_coefs = topic_coefs_pred

predicted_values = []
actual_values = []

for i, doc in enumerate(slda_model.docs):
    topic_dist = doc.get_topic_dist()
    # Ensure topic_dist is 1D and matches topic_coefs
    pred = intercept + sum(coef * prob for coef, prob in zip(topic_coefs, topic_dist))
    predicted_values.append(float(pred))
    actual = float(ipi_values_norm[i])  # Already normalized above
    actual_values.append(actual)

# Convert to 1D numpy arrays for sklearn metrics
#import numpy as np
predicted_values = np.array(predicted_values).flatten() # These are in [0, 1]
# Denormalize:
predicted_values_unnorm = predicted_values * (ipi_max - ipi_min) + ipi_min
actual_values = np.array(actual_values).flatten()

# Calculate performance metrics
mse = mean_squared_error(actual_values, predicted_values)
r2 = r2_score(actual_values, predicted_values)

print(f"Model evaluation: R² = {r2:.4f}, MSE = {mse:.4f}")

Preprocessing text data...


In [ ]:
# After tokenization
num_docs_with_tokens = sum(1 for tokens in docs_tokens if tokens)
print(f"Documents with tokens: {num_docs_with_tokens} / {len(docs_tokens)}")
print("Sample tokenized docs:", docs_tokens[:5])

# After adding documents
print(f"Number of documents in sLDA model: {len(slda_model.docs)}")
assert len(slda_model.docs) > 0, "No documents were added to the sLDA model!"

In [ ]:
# Analyze the topics and regression coefficients
# Extract the top words for each topic
topic_terms = {}
for topic_idx in range(num_topics):
    # Get top 20 words for this topic
    top_terms = slda_model.get_topic_words(topic_idx, top_n=20)
    topic_weight = topic_coefs[topic_idx]  # Get regression coefficient
    topic_terms[topic_idx] = (top_terms, topic_weight)

# Print out the topics and their regression coefficients
print("\nTopic words and regression coefficients (topic influence on IPI):")
for topic_idx, (terms, weight) in topic_terms.items():
    print(f"\nTopic {topic_idx}:")
    print(f"Regression coefficient: {weight:.4f}")
    print(f"Top terms: {', '.join([term for term, _ in terms[:10]])}")
    
# Print topic weights for predicting IPI
print("\nRegression coefficients (topic influence on IPI):")
for k, coef in enumerate(topic_coefs):
    print(f"Topic {k}: {coef:.4f}")

### Visualize Topics

In [ ]:
# Visualize the top topics and their relationship to the IPI values
fig, axes = plt.subplots(2, 5, figsize=(20, 10), constrained_layout=True)
axes = axes.flatten()

for idx, ax in enumerate(axes):
    # Get topic words and their weights
    if idx < num_topics:  # Make sure we don't exceed the number of topics
        # Get the top 50 words for this topic
        top_words = slda_model.get_topic_words(idx, top_n=50)
        
        # Create dictionary for the wordcloud
        topic_dict = {word: weight for word, weight in top_words}
        
        # Create wordcloud
        wc = WordCloud(width=500, height=300, background_color='white', max_words=50)
        wc.generate_from_frequencies(topic_dict)
        ax.imshow(wc, interpolation='bilinear')
        
        # Add title with regression coefficient to show topic-IPI relationship
        coef = topic_coefs[idx]
        coef_sign = "+" if coef > 0 else ""
        ax.set_title(f'Topic {idx + 1} (IPI Effect: {coef_sign}{coef:.3f})', fontsize=14)
        ax.axis('off')
    else:
        ax.axis('off')  # Turn off extra subplots if any

plt.suptitle('Supervised LDA Topics – Word Clouds with IPI Effect', fontsize=18)
plt.show()

In [ ]:
# Save the model
import os

# Create directory if it doesn't exist
os.makedirs("../models/topic_model", exist_ok=True)

# Save the sLDA model using tomotopy's native save method
slda_model.save("../models/topic_model/slda_model")

print("Model saved")

### Topic Extraction & Analysis

In [ ]:
# Step 1: Extract document-topic distributions for each article
doc_topic_dists = []
for doc in slda_model.docs:
    doc_topic_dists.append(doc.get_topic_dist())

# Create topic column names
topic_cols = [f'topic_{i}' for i in range(num_topics)]

# Create a DataFrame with topic distributions
df_topics = pd.DataFrame(doc_topic_dists, columns=topic_cols)

# Combine with metadata
df_meta = df_merged[['date', 'publication', 'month']].reset_index(drop=True).copy()  # Using existing columns
# Only keep rows for which we have topic distributions (some might have been filtered out)
df_meta = df_meta.iloc[:len(doc_topic_dists)].copy()

df_combined = pd.concat([df_meta, df_topics], axis=1)

# Aggregate topic shares by month and publisher
df_monthly_pub = df_combined.groupby(['month', 'publication'])[topic_cols].mean().reset_index()

# Save to CSV with tomotopy_sLDA in the filename
df_monthly_pub.to_csv('../data/processed/monthly_topic_shares_by_publisher_sLDA.csv', index=False)

print("Saved: 'monthly_topic_shares_by_publisher_sLDA.csv'")

# Display the head of the saved CSV file
df_check_csv = pd.read_csv('../data/processed/monthly_topic_shares_by_publisher_sLDA.csv')
print("Head of 'monthly_topic_shares_by_publisher_sLDA.csv':")
df_check_csv.head()

### Compare topic trends with IPI values

In [ ]:
# Analyze the relationship between topic prevalence and IPI values

# Get average IPI by month
monthly_ipi = df_ipi.groupby('month')['ipi_value'].mean().reset_index()

# Get average topic distributions by month (across all publishers)
monthly_topics = df_monthly_pub.groupby('month')[topic_cols].mean().reset_index()

# Merge the data
df_trends = pd.merge(monthly_topics, monthly_ipi, on='month', how='inner')

# Plot the trends of the top 3 most influential topics (based on regression coefficients)
topic_importance = [(i, abs(c)) for i, c in enumerate(topic_coefs)]
top_topics = sorted(topic_importance, key=lambda x: x[1], reverse=True)[:3]

plt.figure(figsize=(12, 8))

# Plot IPI values
ax1 = plt.gca()
ax2 = ax1.twinx()

months = df_trends['month'].astype(str).tolist()
ax1.plot(months, df_trends['ipi_value'], 'k-', label='IPI Value')

# Plot topic trends
for topic_idx, importance in top_topics:
    topic_col = f'topic_{topic_idx}'
    coef = topic_coefs[topic_idx]
    coef_sign = "+" if coef > 0 else ""
    ax2.plot(months, df_trends[topic_col], '--', 
            label=f'Topic {topic_idx} (IPI Effect: {coef_sign}{coef:.3f})')

ax1.set_xlabel('Month')
ax1.set_ylabel('IPI Value')
ax2.set_ylabel('Topic Prevalence')
ax1.set_xticklabels(months, rotation=90)
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

plt.title('IPI Values and Top Topic Trends Over Time')
plt.tight_layout()
plt.show()

### Output

The output CSV file (`monthly_topic_shares_by_publisher_sLDA.csv`) contains the monthly topic distributions for each publisher, which can be used for further analysis or visualization.